## Resnet-18 Code Implementation

### Importing Libraries

In [1]:
import torch
import torchvision.models as models

### Setting up Resnet-18
Weights are from https://www.kaggle.com/datasets/khoongweihao/pytorch-resnet18-34-50-101-152-pretrained?select=resnet18.pth

In [2]:
resnet18 = models.resnet18() 

weights_path = 'resnet18.pth'
resnet18.load_state_dict(torch.load(weights_path)) #load pre-trained model weights

resnet18.eval() #set model to evaluation mode
print(resnet18) #verify model architecture

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Working with pretrained model

In [3]:
import torchvision.transforms as transforms
from torch.nn import Module
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
#import seaborn as sns
import pandas as pd
import numpy as np
import cv2
import os

from flicker_img_whole import create_folder_gif

# Extract all convolutional layers
class ActivationModel(Module):
    def __init__(self, model):
        super(ActivationModel, self).__init__()
        self.features = list(model.children())[:-2]  # Extract all layers except final FC layers
        self.model = torch.nn.Sequential(*self.features)
    
    def forward(self, x):
        activations = []
        for layer in self.model:
            x = layer(x)
            activations.append(x)
        return activations

# Preprocessing for ResNet-18
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), #for converting the PIL Image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
"""
#example usage:
image_path = "durov.jpg"  # Ensure the path is correct
frame = cv2.imread(image_path)
img_array = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert OpenCV BGR to PIL RGB
img = Image.fromarray(img_array)
x = preprocess(img).unsqueeze(0)  # Add batch dimension
"""


def _get_activations(model, frames):
    activations = []
    for frame in frames:
        img_array = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img_array)
        x = preprocess(img).unsqueeze(0)  # Add batch dimension
        with torch.no_grad():
            layer_activations = model(x)  # Forward pass through the model
        activations.append(layer_activations)
    return activations

def save_activations(activations, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for i, frame_activations in enumerate(activations):
        for layer_idx, layer_activation in enumerate(frame_activations):
            np.save(os.path.join(output_dir, f'frame_{i}_layer_{layer_idx}.npy'), layer_activation.cpu().numpy())

# Example usage

# frames = flicker_img('durov.jpg', 5, 'output_animation_whole.gif')
# activation_model = ActivationModel(resnet18)
# activations = _get_activations(activation_model, frames)
# save_activations(activations, 'activations_output')


TypeError: create_folder_gif() missing 1 required positional argument: 'fq2'

In [ ]:
import os
import matplotlib.pyplot as plt

def plot_activations(activations, output_dir, layer_number):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for frame_idx, frame_activations in enumerate(activations):
        layer_activation = frame_activations[layer_number].cpu().numpy()
        layer_activation = layer_activation.squeeze(0)  # Remove batch dimension (1, 64, 112, 112) -> (64, 112, 112)
        ##plot the activation of entire filter.
        #for filter_idx in range(layer_activation.shape[0]):
        ##plot just 10 filters
        for filter_idx in range(0):
            # Create directories for layer and filter
            layer_dir = os.path.join(output_dir, f'layer_{layer_number}')
            filter_dir = os.path.join(layer_dir, f'filter_{filter_idx}')
            if not os.path.exists(filter_dir):
                os.makedirs(filter_dir)

            plt.figure(figsize=(20, 20))
            plt.title(f'Frame {frame_idx}, Layer {layer_number}, Filter {filter_idx}')
            plt.imshow(layer_activation[filter_idx], cmap='viridis')
            plt.axis('off')
            save_path = os.path.join(filter_dir, f'frame_{frame_idx}.png')
            plt.savefig(save_path)
            plt.close()

# Example usage
"""
plot_activations(activations, "frame_plots", layer_number=4)
"""

'\nplot_activations(activations, "frame_plots", layer_number=4)\n'

In [ ]:
from scipy.signal import butter, filtfilt

def highpass_filter(data, cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    y = filtfilt(b, a, data)
    return y

In [ ]:
"""
This is mean activation:
def normalize_activations(activations, num_filters):
    all_activations = np.array([layer_activation[:, :, :num_filters].mean(axis=(1, 2)) for layer_activation in activations])
    mean_activations = all_activations.mean(axis=0)
    std_activations = all_activations.std(axis=0)

    normalized_activations = [(layer_activation[:, :, :num_filters] - mean_activations) for layer_activation in activations] #omitted division by std dev
    return normalized_activations
"""
""" Normalize activations using median 
def normalize_activations(activations, num_filters):
    
    all_activations = np.array([np.median(layer_activation[:, :, :num_filters], axis=(1, 2)) for layer_activation in activations])
    
    median_activations = np.median(all_activations, axis=0)
    
    normalized_activations = [(layer_activation[:, :, :num_filters] - median_activations) for layer_activation in activations]

    return normalized_activations
"""
""" Normalize using Max """
def normalize_activations(activations, num_filters):
    all_activations = np.array([np.max(layer_activation[:, :, :num_filters], axis=(1, 2)) for layer_activation in activations])
    
    max_activations = np.max(all_activations, axis=0)
    
    normalized_activations = [(layer_activation[:, :, :num_filters] - max_activations) for layer_activation in activations]

    return normalized_activations
def normalize_activations_by_filter(activations):
    filter_mean = activations.mean()
    filter_std = activations.std()
    
    normalized_activations = (activations - filter_mean) #omitted division by std dev
    return normalized_activations

In [ ]:
def frequency_analysis(activations, output_dir, layer_number, num_filters=64, fps=30,cutoff=1):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    layer_activations = [frame_activations[layer_number].cpu().numpy() for frame_activations in activations]
    num_frames = len(layer_activations)
    
    # Normalize activations
    #layer_activations = normalize_activations(layer_activations, num_filters)

    plt.figure(figsize=(10, 5))
    
    for filter_idx in range(num_filters):
        #filter_activations = np.array([
            #layer_activation[filter_idx].mean() for layer_activation in layer_activations
        #])

        filter_activations = np.array([
            layer_activation.squeeze().mean()
            for layer_activation in layer_activations
        ])
        
        #apply highpass filter
        filter_activations = highpass_filter(filter_activations, cutoff=cutoff, fs=fps)

        # Normalize activations by filter
        #filter_activations = normalize_activations(filter_activations,num_filters)

        # Compute FFT
        fft_result = np.fft.fft(filter_activations)
        freqs = np.fft.fftfreq(num_frames, d=1/fps)
        
        # Plot frequency spectrum
        plt.plot(freqs[0:num_frames // 2], np.abs(fft_result)[0:num_frames // 2], label=f'Filter {filter_idx}')
    
    plt.title(f'Frequency Analysis - Layer {layer_number}')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.grid(True)
    plt.legend()
    save_path = os.path.join(output_dir, f'layer_{layer_number}_frequency.png')
    plt.savefig(save_path)
    plt.close()

# Example usage
# Assuming `activations` is already defined and contains the activations for each frame
#requency_analysis(activations, "frequency_analysis", layer_number=4, num_filters=10, fps=30,cutoff=1)

## Using the functions we defined

In [ ]:
# Create a new model that outputs the activations of all convolutional layers
activation_model = ActivationModel(resnet18)

# Get activations for each frame generated by flickering the image
frames =create_folder_gif("imgs", 5,6, 'output_animation_whole.gif' ,image_duration=1, fps=30, resize_dim=(640,480))
activations = _get_activations(activation_model, frames)
save_activations(activations, 'activations_output')

#plot_activations(activations, "frame_plots", layer_number=6)

Found 7 image files.
Processing image: imgs/DSC_0545.JPG
Image loaded: imgs/DSC_0545.JPG
Image dimensions: 4928x3264
Number of frames generated: 30
Processing image: imgs/DSC_0890.JPG
Image loaded: imgs/DSC_0890.JPG
Image dimensions: 4928x3264
Number of frames generated: 30
Processing image: imgs/DSC_0895.JPG
Image loaded: imgs/DSC_0895.JPG
Image dimensions: 4928x3264
Number of frames generated: 30
Processing image: imgs/DSC_0903.JPG
Image loaded: imgs/DSC_0903.JPG
Image dimensions: 4928x3264
Number of frames generated: 30
Processing image: imgs/DSC_0909.JPG
Image loaded: imgs/DSC_0909.JPG
Image dimensions: 4928x3264
Number of frames generated: 30
Processing image: imgs/ERHAN_HOCA.jpg
Image loaded: imgs/ERHAN_HOCA.jpg
Image dimensions: 418x494
Number of frames generated: 30
Processing image: imgs/cat_eye_sick.JPG
Image loaded: imgs/cat_eye_sick.JPG
Image dimensions: 4928x3264
Number of frames generated: 30
Final GIF saved: output_animation_whole.gif


In [ ]:
frequency_analysis(activations, "frequency_analysis_hsv_median", layer_number=1)
frequency_analysis(activations, "frequency_analysis_hsv_median", layer_number=2)
frequency_analysis(activations, "frequency_analysis_hsv_median", layer_number=3)
frequency_analysis(activations, "frequency_analysis_hsv_median", layer_number=4)
frequency_analysis(activations, "frequency_analysis_hsv_median", layer_number=5)
frequency_analysis(activations, "frequency_analysis_hsv_median", layer_number=6)
frequency_analysis(activations, "frequency_analysis_hsv_median", layer_number=7)

